In [29]:
import pandas as pd
import spacy
import ast
import re
from pprint import pprint
import json
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from sklearn import linear_model
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, r2_score, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from scipy import sparse
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'good', 'nice', 'dont'])

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yubozhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
reviews = []
with open('scraped_reviews_spotify 2.txt', 'r') as f: #, encoding='utf-8'
    reviews = f.read()

reviews = re.sub('\]', '', reviews)
reviews = re.sub('\[', '', reviews)
reviews = re.sub('\n', '', reviews)
reviews = re.sub("'", '"', reviews)
reviews = re.sub('    ', ' ', reviews)
reviews = re.sub(' {', '{', reviews)
reviews = re.sub(' }', '}', reviews)
reviews = re.sub('{ ', '{', reviews)
reviews = re.sub('} ', '}', reviews)
reviews = re.sub('}  {', '}, {', reviews)
reviews = re.sub('}{', '}, {', reviews)
reviews = re.sub('id:', '"id":', reviews)
reviews = re.sub('userName:', '"userName":', reviews)
reviews = re.sub('userImage:', '"userImage":', reviews)
reviews = re.sub('date:', '"date":', reviews)
reviews = re.sub('url:', '"url":', reviews)
reviews = re.sub('score:', '"score":', reviews)
reviews = re.sub('title:', '"title":', reviews)
reviews = re.sub('text:', '"text":', reviews)
reviews = re.sub('replyDate:', '"replyDate":', reviews)
reviews = re.sub('replyText:', '"replyText":', reviews)
reviews = re.sub(': undefined', ': "undefined"', reviews)

In [31]:
review_strings = [r+'}' for r in reviews.split('},')]
review_strings[-1] = review_strings[-1][:-1]

In [32]:
review_list = []
for r in review_strings:
    try:
        review_list.append(json.loads(r))
    except:
        pass

In [33]:
raw_corpus = []
for i in range(1000): # change to larger number
    raw_corpus.append(review_list[i][u'text'])

In [34]:
data = raw_corpus

In [35]:
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub("\"", "", sent) for sent in data]
data = [re.sub("_", "", sent) for sent in data]

data = [re.sub("good'", "", sent) for sent in data]
data = [re.sub("app", "", sent) for sent in data]
data = [re.sub("nice", "", sent) for sent in data]
pprint(data[1:10])


[u'Collection',
 u'Reliable lication for team projects',
 u'Great  we use at work',
 u'Take too much time to load.Expecting a lighter version soon',
 u'Great',
 u'The latest update not at all works. Always shows opening. What is this?',
 u'It works like a charm',
 u'Awesome ',
 u'33 MB for a chat ?']


In [36]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str([sentence]), deacc=True)) 


In [37]:
data_words = list(sent_to_words(data))

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
#print(trigram_mod[bigram_mod[data_words[0][:30]]])
#print(list(sent_to_words(data)))

In [38]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        if sent != []:
            doc = nlp(" ".join(sent)) 

            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
 
    return texts_out

In [39]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized)

[[u'waste'], [u'collection'], [u'reliable', u'lication', u'team', u'project'], [u'great', u'work'], [u'take', u'much', u'time', u'load', u'expect', u'light', u'version', u'soon'], [u'great'], [u'late', u'update', u'work', u'always', u'show', u'opening'], [u'work', u'charm'], [u'awesome'], [u'chat'], [u'functionality', u'bit', u'complicated'], [u'best', u'useful'], [u'love', u'phone', u'integrate', u'perfectly', u'desktop', u'easy', u'stay', u'touch', u'team', u'great', u'way', u'make', u'sure', u'everyone', u'include', u'conversation'], [u'amazing'], [u'favorite', u'communication', u'app', u'company'], [u'make', u'feel', u'connect', u'real', u'time', u'feel', u'pressured', u'engage'], [u'great', u'think', u'company', u'could', u'run', u'slack', u'point'], [u'great', u'clicking', u'view', u'reminder', u'permanently', u'freeze', u'require'], [u'communication', u'team', u'small', u'organisation', u'experience', u'big', u'company'], [u'easy'], [u'awesome'], [u'want', u'wallet', u'that', u'

In [40]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus)

[[(0, 1)], [(1, 1)], [(2, 1), (3, 1), (4, 1), (5, 1)], [(6, 1), (7, 1)], [(8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(6, 1)], [(7, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)], [(7, 1), (21, 1)], [(22, 1)], [(23, 1)], [(24, 1), (25, 1), (26, 1)], [(27, 1), (28, 1)], [(5, 1), (6, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)], [(43, 1)], [(44, 1), (45, 1), (46, 1), (47, 1)], [(14, 1), (36, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1)], [(6, 1), (46, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1)], [(6, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1)], [(5, 1), (45, 1), (46, 1), (64, 1), (65, 1), (66, 1), (67, 1)], [(31, 1)], [(22, 1)], [(68, 1), (69, 1), (70, 1), (71, 1), (72, 2)], [(73, 1), (74, 1)], [(75, 1)], [(76, 1)], [(77, 1), (78, 1), (79, 1), (80, 1), (81, 1)], [(82, 1)], [(5, 1), (6, 1), (83, 1), (84, 1)], [(22, 1), (85, 1), (86, 1), (87, 1), (88, 1),

In [41]:
id2word[1]

u'collection'

In [42]:
# Human readable format of corpus (term-frequency)
#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [43]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [44]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  u'0.055*"song" + 0.054*"great" + 0.028*"music" + 0.026*"play" + 0.024*"playlist" + 0.023*"spotify" + 0.020*"listen" + 0.020*"not" + 0.020*"get" + 0.018*"make"'),
 (1,
  u'0.061*"awesome" + 0.031*"thank" + 0.026*"update" + 0.017*"still" + 0.017*"old" + 0.017*"fix" + 0.014*"bug" + 0.014*"pandora" + 0.012*"everything" + 0.011*"video"'),
 (2,
  u'0.042*"amazing" + 0.033*"need" + 0.032*"add" + 0.023*"offline" + 0.022*"always" + 0.014*"away" + 0.014*"rock" + 0.013*"useful" + 0.013*"feature" + 0.011*"hit"'),
 (3,
  u'0.171*"love" + 0.101*"music" + 0.061*"good" + 0.034*"spotify" + 0.020*"work" + 0.017*"day" + 0.017*"selection" + 0.016*"favorite" + 0.015*"pretty" + 0.010*"well"')]


In [45]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      53.387581        1       1 -0.278753 -0.070900
3      22.062065        1       2  0.015653  0.252552
2      12.609185        1       3  0.105576 -0.094276
1      11.941169        1       4  0.157524 -0.087376, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
132   Default  173.000000         love  173.000000  30.0000  30.0000
158   Default  170.000000        music  170.000000  29.0000  29.0000
314   Default   62.000000         good   62.000000  28.0000  28.0000
31    Default  134.000000         song  134.000000  27.0000  27.0000
112   Default  133.000000        great  133.000000  26.0000  26.0000
213   Default   33.000000      awesome   33.000000  25.0000  25.0000
235   Default   24.000000      amazing   24.000000  24.0000  24.0000
295   Default   90.000000      spotify   90.000000  23.0000  23.0000
495   Default   19.000000         need   19.000000  22.0000  22.0000
845   Default   19.000000          add   19.000000  21.0000  21.0000
298   Default   17.000000        thank   17.000000  20.0000  20.0000
515   Default   64.000000         play   64.000000  19.0000  19.0000
1062  Default   15.000000       update   15.000000  18.0000  18.0000
192   Default   20.000000         work   20.000000  17.0000  17.0000
374   Default   13.000000      offline   13.000000  16.0000  16.0000
335   Default   13.000000       always   13.000000  15.0000  15.0000
329   Default   18.000000          day   18.000000  14.0000  14.0000
450   Default   50.000000       listen   50.000000  13.0000  13.0000
326   Default   50.000000          not   50.000000  12.0000  12.0000
555   Default   17.000000    selection   17.000000  11.0000  11.0000
433   Default   60.000000     playlist   60.000000  10.0000  10.0000
939   Default   16.000000     favorite   16.000000   9.0000   9.0000
781   Default   45.000000         make   45.000000   8.0000   8.0000
583   Default   15.000000       pretty   15.000000   7.0000   7.0000
291   Default   10.000000        still   10.000000   6.0000   6.0000
890   Default   10.000000          old   10.000000   5.0000   5.0000
592   Default   50.000000          get   50.000000   4.0000   4.0000
688   Default    9.000000          fix    9.000000   3.0000   3.0000
30    Default   39.000000      premium   39.000000   2.0000   2.0000
210   Default   37.000000         want   37.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
298    Topic4   16.818581        thank   17.469307   2.0872  -3.4816
1062   Topic4   14.486027       update   15.135530   2.0813  -3.6309
688    Topic4    9.309843          fix    9.949679   2.0587  -4.0730
291    Topic4    9.524041        still   10.187255   2.0579  -4.0502
890    Topic4    9.487668          old   10.176536   2.0551  -4.0541
1002   Topic4    7.830722          bug    8.478433   2.0457  -4.2460
815    Topic4    7.692608      pandora    8.416525   2.0352  -4.2638
917    Topic4    6.359413   everything    7.006152   2.0283  -4.4541
233    Topic4    5.740075        write    6.403993   2.0157  -4.5566
76     Topic4    6.214957        video    6.955043   2.0127  -4.4771
945    Topic4    5.304160       mantap    5.950249   2.0102  -4.6356
4      Topic4    5.607898         lack    6.295033   2.0096  -4.5799
851    Topic4    5.324178         save    5.980316   2.0090  -4.6318
75     Topic4    5.200015       second    5.869264   2.0041  -4.6554
674    Topic4    5.099205         last    5.810134   1.9947  -4.6750
160    Topic4    4.800198       memory    5.480662   1.9926  -4.7354
850    Topic4    4.635184        sweet    5.295808   1.9919  -4.7704
1012   Topic4    4.635184     soundful    5.295808   1.9919  -4.7704
383    Topic4    4.635183       friday    5.295809   1.9919  -4.7704
612    Topic4    4.635183       freaky    5.2

In [23]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


('\nPerplexity: ', -6.318845249752648)
('\nCoherence Score: ', 0.4574147550313077)
